# 3章 正規表現

## 20.JSONデータの読み込み

In [ ]:
import json
with open("jawiki-country.json") as f:
    for line in f:
        data_json = json.loads(line)
        if data_json['title'] =='イギリス':
            print(data_json['text'])
            break

## 21.カテゴリ名を含む行を抽出

In [ ]:
import re
answer =[]
for line in data_json['text'].split("\n"):
    matched_string = re.search('\[\[Category:(.*)?\]\]',line)
    if matched_string:
        answer.append(matched_string[0])

for ans in answer:
    print(ans)

## 22.カテゴリ名の抽出

In [ ]:
answer = []
for line in data_json['text'].split("\n"):
    matched_string = re.search('\[\[Category:(.*)?\]\]',line)
    if matched_string:
        string = matched_string.group(1)
        change_str = re.sub('\|\*',"",string)
        answer.append(change_str)
for ans in answer:
    print(ans)

## 23.セクション構造

In [ ]:
lines = data_json['text'].split("\n")
for line in lines:
    if re.match("==",line):
        print(re.sub("=","",line),line.count("=")//2 -1)

## 24.ファイル参照

In [ ]:
answer =[]
lines = data_json['text'].split("\n")
for line in lines:
    if re.match("\[\[ファイル:[^\]]+\]\]",line):
        answer.append(line)
        
for ans in answer:
    print(re.sub("[(^(\[\[ファイル:))|(\]\])]","",ans))

## 25.テンプレートの抽出

In [ ]:
dic = {}
long_string = data_json['text']
string = re.search("\{\{基礎情報(.*?)\n}}",long_string,flags=re.DOTALL)
word_list = string.group(0).split("\n|")
for line in word_list[1:]:
    res = re.search("(?P<key>.*?)=(?P<value>.*)",line)
    dic[res.group('key')]= res.group('value')
    
for key in dic :
    print(key,"====",dic[key])

## 26.強調マークアップの除去

In [ ]:
dic = {}
long_string = data_json['text']
long_string = re.sub("'{2,}(.*?)'{2,}","\\1",long_string)
string = re.search("\{\{基礎情報(.*?)\n}}",long_string,flags=re.DOTALL)
word_list = string.group(0).split("\n|")
for line in word_list[1:]:
    res = re.search("(?P<key>.*?)=(?P<value>.*)",line)
    dic[res.group('key')]= res.group('value')
    
for key in dic :
    print(key,"====",dic[key])

## 27.内部リンクの除去

In [ ]:
dic = {}
long_string = data_json['text']
long_string = re.sub("'{2,}(.*?)'{2,}","\\1",long_string)
long_string = re.sub("\[\[[^\[\]]*\|(.*?)\]\]","\\1",long_string)
long_string = re.sub("\[\[(.*?)\]\]","\\1",long_string)
string = re.search("\{\{基礎情報(.*?)\n}}",long_string,flags=re.DOTALL)
word_list = string.group(0).split("\n|")
for line in word_list[1:]:
    res = re.search("(?P<key>.*?)=(?P<value>.*)",line)
    dic[res.group('key')]= res.group('value')
    
for key in dic :
    print(key,"====",dic[key])

## 28.MediaWikiマークアップの除去

In [ ]:
dic = {}
long_string = data_json['text']
long_string = re.sub("'{2,}(.*?)'{2,}","\\1",long_string)
long_string = re.sub("\[\[[^\[\]]*\|(.*?)\]\]","\\1",long_string)
long_string = re.sub("\[\[(.*?)\]\]","\\1",long_string)
#########################
long_string = re.sub("\[(.*?)\]",'',long_string)
long_string = re.sub("<.*?>","",long_string)
long_string = re.sub("\{\{(.*?)\}\}","",long_string)
#########################
string = re.search("\{\{基礎情報(.*?)\n}}",long_string,flags=re.DOTALL)
word_list = string.group(0).split("\n|")
for line in word_list[1:]:
    res = re.search("(?P<key>.*?)=(?P<value>.*)",line)
    dic[res.group('key')]= res.group('value')
    
for key in dic :
    print(key,"====",dic[key])

## 29.国旗画像のURLを取得する

In [ ]:
import requests

dic = {}
long_string = data_json['text']
long_string = re.sub("'{2,}(.*?)'{2,}","\\1",long_string)
long_string = re.sub("\[\[[^\[\]]*\|(.*?)\]\]","\\1",long_string)
long_string = re.sub("\[\[(.*?)\]\]","\\1",long_string)
long_string = re.sub("\[(.*?)\]",'',long_string)
long_string = re.sub("<.*?>","",long_string)
long_string = re.sub("\{\{(.*?)\}\}","",long_string)
string = re.search("\{\{基礎情報(.*?)\n}}",long_string,flags=re.DOTALL)
word_list = string.group(0).split("\n|")
for line in word_list[1:]:
    res = re.search("(?P<key>.*?)\s*=\s*(?P<value>.*)",line,flags=re.DOTALL)
    dic[res.group('key')]= res.group('value')

res = requests.get("https://en.wikipedia.org/w/api.php",
                   params={"action":"query",
                           "titles":"File:"+dic['国旗画像'],
                           "prop":"imageinfo",
                           "format":"json",
                           "iiprop":"url"})


In [ ]:
word = res.json()
word['query']['pages']['23473560']['imageinfo'][0]['url']